In [20]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.constraints import maxnorm
from keras.optimizers import SGD

from IPython.display import SVG,display
from keras.utils.vis_utils import model_to_dot

batch_size = 32
num_classes = 11
epochs = 1

In [18]:
def hot(y):
    label_binarizer = LabelBinarizer()
    label_binarizer.fit(range(11))
    new_y = np.zeros((y.shape[0],y.shape[1]*num_classes))
    for i in range(len(y)):
        new_y[i,:] = label_binarizer.transform(y[i]).flatten()
    return new_y

In [14]:
X = np.load("modimages3.npy")
y = np.load("mergetrain.npy")
y = np.delete(y,-1,1).astype(int)
y = hot(y)
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42)

In [9]:
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')

x_train shape: (30061, 28, 74, 3)
y_train shape: (30061, 55)
y_train shape: (30061, 55)
30061 train samples


In [22]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=x_train[1,:].shape, activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(num_classes*5, activation='softmax'))

# load weights
#model.load_weights("weights.best.hdf5")

# Compile model
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

print(model.summary())

#display(SVG(model_to_dot(model).create(prog='dot', format='svg')))

x_train = x_train.astype('float32')
x_train /= 255

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 28, 74, 32)        896       
_________________________________________________________________
dropout_25 (Dropout)         (None, 28, 74, 32)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 28, 74, 32)        9248      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 14, 37, 32)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 14, 37, 64)        18496     
_________________________________________________________________
dropout_26 (Dropout)         (None, 14, 37, 64)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 14, 37, 64)        36928     
__________

In [24]:
# checkpoint
filepath="weights.best.hdf5"
checkpointer = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpointer]

In [27]:
model.fit(x_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(x_test, y_test),
        shuffle=True,
        callbacks=[checkpointer])

Train on 30061 samples, validate on 3341 samples
Epoch 1/2
 2464/30061 [=>............................] - ETA: 40s - loss: 14.1434 - acc: 0.0000e+00

KeyboardInterrupt: 